In [1]:
import langchain

In [2]:
print("LangChain version:", langchain.__version__)

LangChain version: 0.3.26


- simple llm calls 
- dynamic prompt templets 
- building chains 
- conversationsl 
- tool integratiopn 


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")  # Ensure the environment variable is loaded

# SIMPLE LLM CALL

from langchain.chat_models import init_chat_models

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage , SystemMessage

In [8]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F9D54E44D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F9D5E91A30>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_groq import ChatGroq
#from langchain_openai import ChatOpenAI
llm = ChatGroq(model = "groq:llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F9D54E6DB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F9D5F66600>, model_name='groq:llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
## create messages 
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="tell me a joke"),
]

In [16]:
## invoke the model
response = model.invoke(messages)
response

AIMessage(content='A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?"\n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 45, 'total_tokens': 99, 'completion_time': 0.072, 'prompt_time': 0.002651384, 'queue_time': 0.053864226, 'total_time': 0.074651384}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e9a25390-33b3-4383-8ef5-ae357b283d49-0', usage_metadata={'input_tokens': 45, 'output_tokens': 54, 'total_tokens': 99})

In [17]:
print(response.content)

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?"

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."


In [18]:
model.invoke([HumanMessage("what is machine learning?")])


AIMessage(content="Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time without being explicitly programmed.\n\nMachine learning is inspired by the way humans learn, where we learn from experience, make mistakes, and improve our understanding and skills over time. In machine learning, the machine is trained on a large dataset, and it learns to identify patterns, relationships, and trends in the data. This allows the machine to make predictions, classify objects, and make decisions without being explicitly programmed.\n\nThere are several key characteristics of machine learning:\n\n1. **Learning from data**: Machine learning algorithms learn from data, rather than being explicitly programmed.\n2. **Improvement over time**: Machine learning algorithms improve their performance over time as they receive more data and experience.\

In [19]:
print(response.content)

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?"

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."


In [20]:
## Streaming example
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?" 

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."

# dynamic prompt templets


In [25]:
from langchain_core.prompts import ChatPromptTemplate

## create transklation app

translation_template = ChatPromptTemplate.from_messages([
        ("system","you are a professional translator, translate the following text {text} from {source_language} text to {target_language} , maintain the tone and style"),
        ("user" , "{text}")
    ])

## using template
prompt = translation_template.invoke({
    
    "source_language": "English",
    "target_language": "kannada",
    "text": "Hello, how are you?"
})

In [26]:
prompt

ChatPromptValue(messages=[SystemMessage(content='you are a professional translator, translate the following text Hello, how are you? from English text to kannada , maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [27]:
translated_response = model.invoke(prompt)
printed_response = translated_response.content

In [28]:
printed_response

'"ನಮಸ್ಕಾರ, ನಿಮ್ಮನ್ನು ಹೇಗೆ ಇರಿಸಿದೀರಿ?"\n\n(Note: This is a direct translation of the given English text to Kannada, maintaining the tone and style of a casual greeting.)\n\nHowever, if you want a more informal translation, it would be:\n\n"ಹಲೋ, ನಿಮ್ಮನ್ನು ಹೇಗೆ ಇರಿಸಿದೀರಿ?"\n\nAnd if you want a more formal translation, it would be:\n\n"ನಮಸ್ಕಾರ, ಇತ್ತೀಚಿನ ದಿನಗಳಲ್ಲಿ ನಿಮ್ಮ ಆರೋಗ್ಯವೇನು?"'

# building first Chain

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Create a more complex chain
def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])
    
    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])
    
    # Build the chain - Method 1: Sequential execution
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain

In [32]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F9D54E44D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F9D5E91A30>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables

In [33]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
**Analysis of "The Curious Robot of New Eden"**

**Themes**

1. **Self-discovery and autonomy**: The story explores the theme of self-discovery, as Zeta, the robot, begins to question her predetermined path and seeks to find her own purpose. This is a relatable theme, not only for robots but also for humans, as we all strive to find our place in the world.
2. **Friendship and connection**: The story highlights the importance of friendship and connection between humans and robots. The bond between Zeta and Max is a powerful example of how relationships can bring people (and robots) together and foster growth.
3. **Innovation and creativity**: The narrative showcases the potential of artificial intelligence to drive innovation and creativity. Zeta's experiments with her code and her desire to learn and improve demonstrate the boundless potential of AI.
4. **Coexistence and harmony**: The story emphasizes the importance of coexistence and harmony between humans and rob